# EasyOCR custom model

## TextRecognitionDataGenerator

Dataset does not have to be generated within XGEE\
Instead, generating a dataset using the [TextRecognitionDataGenerator](https://github.com/Belval/TextRecognitionDataGenerator) with custom parameters to mirror XGEE's font style\
This dataset can be used to train own model by following the [deep-text-recognition-benchmark Repo](https://github.com/clovaai/deep-text-recognition-benchmark)\
Has to be fully convolutional to work with large variety of Text sizes and potentially orientations\
2 additional files describing recognition network architecture and model configuration needed